## Section 0

In [ ]:
!nvidia-smi

Fri Aug 30 03:40:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q -U langchain transformers bitsandbytes accelerate langchain_community

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Section 1

In [ ]:
quantization_config_1 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_1 = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map="auto", quantization_config=quantization_config_1)
tokenizer_1 = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Section 2

In [ ]:
# Import Dataset 1

# data path
data_path = '/content/final_dataset.csv'

# read a csv file
dataset = pd.read_csv(data_path)

# display
dataset

,subject,body,subject_length,body_length,special_symbol_number,average_word_length,url_average_length,is_top_website,is_dga,sentiment_score,...,num_authority,num_urgency,num_curiosity,num_familiarity,num_emotional,num_sensitive,num_social_proof,subject_entropy,body_entropy,label
0,Re: [Python-3000] [Python-Dev] Reminder: last ...,"On 01:55 am, hoauf@python.org wrote:\n>On Thu,...",79,12215,184,4.91,63.5,False,False,Positive,...,74,6,10,2,17,0,0,4.68,4.63,0
1,terminated employees ' benefits,"happy new year all ,\r\ni have been with enron...",31,1670,7,3.80,0.0,True,False,Positive,...,2,2,4,1,2,1,0,3.76,4.17,0
2,"9/11, war in Iraq threaten Disney parks",URL: http://boingboing.net/#85531557\nDate: No...,39,1044,12,6.76,58.0,False,False,Positive,...,0,0,2,1,1,0,0,4.03,4.89,0
3,"If you want a decent watch, get a replica appr...",\nQualitative watches at Replica Classics \n\n...,58,187,0,6.04,25.0,False,False,Positive,...,0,0,0,0,0,0,0,3.88,4.43,1
4,re : tds project,"you might hear of this , so i thought i better...",16,6807,98,3.71,0.0,True,False,Positive,...,15,5,2,0,7,0,0,3.33,4.43,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39921,caiso notification - tswg conference call,please call in @ 1330 pacific . call in number...,41,84,4,3.42,0.0,True,False,Positive,...,0,0,0,0,0,0,0,3.71,4.42,0
39922,[UAI] Ph.D Positions in Bioinformatics at Joha...,Ph.D Positions in Bioinformatics\n\nTwo PhD po...,82,2000,11,6.24,0.0,True,False,Positive,...,4,1,1,0,2,0,0,4.45,4.64,0
39923,happy holidays !,with the holiday season and a new year upon us...,16,386,0,4.11,0.0,True,False,Positive,...,0,0,2,1,2,0,0,3.38,4.16,0
39924,Re: [opensuse] Screen Capture,"Chris Arnold wrote:\n> On 10.3, how do i get a...",29,591,11,4.63,0.0,True,False,Negative,...,0,0,1,0,1,0,0,3.81,4.63,0


In [ ]:
begin_number = 29700
end_number = 30000

small_dataset = dataset.iloc[begin_number:end_number, :20]

small_label = dataset.iloc[begin_number:end_number, 20]

In [ ]:
small_dataset
body_length = small_dataset['body'].apply(len)

In [ ]:
def check_body_length(body_length_0, threshold):
    index_num = 0
    index_dict_check = {}
    index_list_check = []
    max_length_00 = 0

    for length in body_length:
        if length > threshold:
            index_dict_check[index_num] = length
            index_list_check.append(index_num)

        if length > max_length_00:
            max_length_00 = length

        index_num += 1

    return index_dict_check, index_list_check, max_length_00


dict_00, list_00, max_length = check_body_length(body_length, 50000)

In [ ]:
print(dict_00)
print(max_length)

{}
30991


In [ ]:
small_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 29700 to 29999
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   subject                300 non-null    object 
 1   body                   300 non-null    object 
 2   subject_length         300 non-null    int64  
 3   body_length            300 non-null    int64  
 4   special_symbol_number  300 non-null    int64  
 5   average_word_length    300 non-null    float64
 6   url_average_length     300 non-null    float64
 7   is_top_website         300 non-null    bool   
 8   is_dga                 300 non-null    bool   
 9   sentiment_score        300 non-null    object 
 10  num_scarcity           300 non-null    int64  
 11  num_authority          300 non-null    int64  
 12  num_urgency            300 non-null    int64  
 13  num_curiosity          300 non-null    int64  
 14  num_familiarity        300 non-null    int64  
 15  

## Section 3

In [ ]:
# 存储字符串的列表
strings_list = []

# 迭代数据框的每一行
for index, row in small_dataset.iterrows():
    formatted_string = f"The new inputs is coming: \n Email Content: {row['body']} \n " \
                       f"Email Subject: {row['subject']} \n " \
                       f"Subject Length: {row['subject_length']} \n " \
                       f"Content Length: {row['body_length']} \n " \
                       f"Special Symbol Number: {row['special_symbol_number']} \n " \
                       f"Average Word Length per Sentence: {row['average_word_length']} \n " \
                       f"URL Average Length: {row['url_average_length']} \n " \
                       f"URL Belong to Top Website: {row['is_top_website']} \n " \
                       f"IS DGA: {row['is_dga']} \n " \
                       f"Sentiment Score: {row['sentiment_score']} \n " \
                       f"Number scarcity: {row['num_scarcity']} \n " \
                       f"Number authority: {row['num_authority']} \n " \
                       f"Number urgency: {row['num_urgency']} \n " \
                       f"Number curiosity: {row['num_curiosity']} \n " \
                       f"Number familiarity: {row['num_familiarity']} \n " \
                       f"Number emotional: {row['num_emotional']} \n " \
                       f"Number sensitive: {row['num_sensitive']} \n " \
                       f"Number social proof: {row['num_social_proof']} \n " \
                       f"Subject Entropy: {row['subject_entropy']} \n " \
                       f"Content Entropy: {row['body_entropy']}."
    strings_list.append(formatted_string)

In [ ]:
print(strings_list[0])

The new inputs is coming: 
 Email Content: Justin Piszcz wrote:
>
>
> On Tue, 12 Feb 2008, James wrote:
>
>> This is a stupid question and should be pretty easy for someone to 
>> answer.
>>
>> What's the conf line for setting a blacklist of recipient addresses 
>> in postfix?
>>
>> for instance odgovlxx@mydomain.com mailbox doesn't exist.
>> I want my mailserver to reject that as soon as someone tries to send 
>> to that address.
>>
>> This is a relay server so i don't want to do checking of my mailboxes 
>> i just want to make a simple list of mail addresses to reject 
>> delivery to.
>>
>> I'm thinking it's something like this
>>
>> |hash:/etc/postfix/sender_access|
>>
>> |smtpd_recipient_restrictions =
>>               check_sender_access
>>
>>
>> sender_access:
>> |odgovlxx@mydomain.com   REJECT
>> kqpixrkktgd@mydomain.com   REJECT
>> ywcvgrdlkkcvjoj@mydomain.com   REJECT
>>
>> postmap sender_access
>>
>>
>> Is that right?
>> I just want to check before i start doing anything
>>
>

In [ ]:
print(len(strings_list))

300


In [ ]:
def helper_function_01(input_01_1, dictionary_0, current_number, log, c_number):
    final_content_01 = input_01_1
    lines = final_content_01.strip().split('\n')

    first_element, second_element, third_element, fourth_element = 0, 0, 0, 0

    for raw_line in lines:

        line = raw_line.strip()

        if line.startswith("Email Type"):
            try:
                first_element = line.split(":")[1]
                first_element = first_element.strip()

            except IndexError:
                first_element = 0
                log_message_1 = "Failure: IndexError"

            except Exception as e:
                first_element = 0
                log_message_1 = "Failure: Other Error"

            else:
                log_message_1 = "Successful"

        elif line.startswith("Brand Name"):
            try:
                second_element = line.split(":")[1]
                second_element = second_element.strip()

            except IndexError:
                second_element = 0
                log_message_2 = "Failure: IndexError"

            except Exception as e:
                second_element = 0
                log_message_2 = "Failure: Other Error"

            else:
                log_message_2 = "Successful"


        elif line.startswith("Brand Category"):
            try:
                third_element = line.split(":")[1]
                third_element = third_element.strip()

            except IndexError:
                third_element = 0
                log_message_3 = "Failure: IndexError"

            except Exception as e:
                third_element = 0
                log_message_3 = "Failure: Other Error"

            else:
                log_message_3 = "Successful"


        elif line.startswith("Explanation:"):
            try:
                explanation_start = lines.index(raw_line) + 1
                explanation = ' '.join(lines[explanation_start:])
                fourth_element = explanation.strip()

            except IndexError:
                fourth_element = 0
                log_message_4 = "Failure: IndexError"

            except Exception as e:
                fourth_element = 0
                log_message_4 = "Failure: Other Error"

            else:
                log_message_4 = "Successful"

            break

    if first_element == 0 or second_element == 0 or third_element == 0 or fourth_element == 0:
        error_number_list.append(current_number)
        return False

    if c_number < 10:
        if first_element != "Benign" and first_element != "Phishing" and first_element != "Unknown":
            error_number_list.append(current_number)
            return False

    dictionary_0["Email Type"].append(first_element)
    dictionary_0["Brand Name"].append(second_element)
    dictionary_0["Brand Category"].append(third_element)
    dictionary_0["Explanation"].append(fourth_element)

    log["Number"].append(current_number)
    log["First Element"].append(log_message_1)
    log["Second Element"].append(log_message_2)
    log["Third Element"].append(log_message_3)
    log["Fourth Element"].append(log_message_4)

    return True

## Section 4

In [ ]:
new_dataset_dictionary = {
    "Email Type": [],
    "Brand Name": [],
    "Brand Category": [],
    "Explanation": []
}

log_dictionary = {
    "Number": [],
    "First Element": [],
    "Second Element": [],
    "Third Element": [],
    "Fourth Element": []
}

acc = 0
acc_2 = 0
check_2 = False
error_number_list = []
output_string = []
index_list = []

while acc < len(strings_list):

    new_input_3 = strings_list[acc]

    messages = [
        {"role": "user", "content": "I want to use a Large Language Model (LLM) to detect phishing emails. In the next few prompts, I will provide two template examples that include both input and output. Using these two templates, I will then provide new inputs for which you need to generate the corresponding outputs, identifying whether this input is a phishing email or not based on the template provided."},
        {"role": "assistant", "content": "Sure, I'll do my best to assist you in detecting phishing emails using the Large Language Model (LLM). Please provide these two template examples, including both input and output, and I will generate the corresponding outputs for new inputs based on these templates. \n For the input, please provide a sample email that you would like me to analyze, and for the output, please indicate whether the email is a phishing attempt or not, and explain the reasons for my classification. \n Once you provide the two templates, I will follow your format and generate the corresponding outputs for the new inputs that you provide."},
        {"role": "user", "content": "The first template is in the following. \n Inputs: \n Email Content: >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+= >THE DAILY TOP 10 >from CNN.com >Top videos and stories as of: Aug 1, 2008 3:58 PM EDT >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+= TOP 10 VIDEOS 1. MONTAUK 'MONSTER' http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/07/31/moos.montauk.monster.cnn Is it a devil dog? Is it a turtle? Is it the Montauk Monster? CNN's Jeanne Moos asks, \"what is this thing?\" 2. RACY PHOTOS OF TODDLER'S MOM http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/07/31/ng.racy.photos.cnn 3. NEWS OF THE ABSURD EPISODE 54 http://www.cnn.com/video/partners/email/index.html?url=/video/podcasts/absurd/site/2008/08/01/nota.episode.54.cnn 4. POLICE BEATING DISPUTE http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/07/31/levs.police.video.cnn 5. MOM PLEADS FOR GIRL'S RETURN http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/08/01/hill.boss.reigh.plea.cnn 6. DEFENDANT FAKES HEART ATTACK http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/08/01/dnt.fake.heart.attack.mxf.whio 7. KILLER CARRIED VICTIM'S HEAD http://www.cnn.com/video/partners/email/index.html?url=/video/world/2008/07/31/natpkg.can.bus.decapitation.ctv 8. MURDER CONFESSION RECANTED http://www.cnn.com/video/partners/email/index.html?url=/video/world/2008/08/01/blake.brazil.teen.murder.itn 9. ANTHRAX SUSPECT'S HOME http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/08/01/von.dr.bruce.ivins.home.cnn 10. HECKLERS INTERRUPT OBAMA TALK http://www.cnn.com/video/partners/email/index.html?url=/video/politics/2008/08/01/sot.fl.obama.protesters.baynews9 TOP 10 STORIES 1. SUSPECT IN BEHEADING IDENTIFIED http://www.cnn.com/2008/WORLD/americas/08/01/canada.beheading/index.html Canadian police say Vince Weiguang Li, 40, of Edmonton is charged with second-degree murder in the beheading of a man on a bus. 2. JUDGE TAKEN OFF LAST JENA 6 CASES http://www.cnn.com/2008/CRIME/08/01/jena6.appeal/index.html 3. PEOPLE MAG GETS PITT-JOLIE PIX http://www.cnn.com/2008/SHOWBIZ/Movies/08/01/brangelina.photos.ap/index.html 4. ATTACK IN TORONTO CALLED RACIAL http://www.cnn.com/2008/WORLD/americas/08/01/canada.attack.ap/index.html 5. IREPORTERS' UNUSUAL NAMES http://www.cnn.com/2008/LIVING/08/01/unusual.names.irpt/index.html 6. MOTHER PLEADS FOR CHILD'S RETURN http://www.cnn.com/2008/CRIME/08/01/rockefeller.kidnapping/index.html 7. KARADZIC: I MADE DEAL WITH U.S. http://www.cnn.com/2008/WORLD/europe/08/01/karadzic.trial/index.html 8. SUSPECT ARRESTED IN SWIM KILLINGS http://www.cnn.com/2008/CRIME/08/01/wisconsin.shooting.ap/index.html 9. ANTHRAX SUSPECT APPARENT SUICIDE http://www.cnn.com/2008/CRIME/08/01/anthrax.death/index.html 10. MCCAIN: OBAMA CRITICISM 'FAIR' http://www.cnn.com/2008/POLITICS/08/01/campaign.wrap/index.html CNN, The Most Trusted Name in News > Cable News Network LP, LLLP. < > One CNN Center, Atlanta, Georgia 30303 < > 2008 Cable News Network LP, LLLP. < > A Time Warner Company. < > All Rights Reserved. < ========================================================= = Please send comments or suggestions by going to = = http://www.cnn.com/feedback/ = = = = Read our privacy guidelines by going to = = http://www.cnn.com/privacy.html = ========================================================= You have agreed to receive this email from CNN.com as a result of your CNN.com preference settings. To manage your settings, go to: http://www.cnn.com/linkto/bn.manage.html To unsubscribe from the Daily Top 10, go to http://cgi.cnn.com/m/clik?e=email118@gvc.ceas-challenge.cc&l=cnn-dailytop10 \n Email Subject: CNN.com Daily Top 10 \n Subject Length: 20 \n Content Length: 3894 \n Special Symbol Number: 292 \n Average Word Length per Sentence: 11.08 \n URL Average Length: 79.17 \n URL Belong to Top Website: False \n IS DGA: True \n Sentiment Score: Negative \n Number scarcity: 0 \n Number authority: 11 \n Number urgency: 0 \n Number curiosity: 5 \n Number familiarity: 0 \n Number emotional: 1 \n Number sensitive: 0 \n Number social proof: 0 \n Subject Entropy: 3.88 \n Content Entropy:5.25 \n\n Outputs: \n Email Type: Phishing \n Brand Name: CNN \n Brand Category: News \n Explanation: \n Average URL length: The average length of URLs included in the emails was 79.17, which is unusually long. Generally speaking, legitimate URLs are usually shorter, especially those from well-known websites. \n URLs do not belong to top-level websites: Although the email looks like it comes from CNN, the URLs in it do not belong to CNN's top-level website domain. This is a major red flag and may indicate that these links are fake and designed to trick users into clicking and leaking sensitive information. \n Abnormal URL format: The URLs in the email contain many complex paths and parameters, which is not common in legitimate emails. Such complex URLs are often used to hide malicious code or redirect to phishing websites. \n Negative sentiment score: The email has a negative sentiment score, indicating that its content may induce negative emotions, such as fear or urgency. Phishing emails often use this strategy to lure recipients into clicking malicious links. \n Unusual number of symbols: The email contains 292 special symbols, which is much more than normal emails. This may be to circumvent spam filters or make the email look more formal and complex, thereby increasing credibility. \n Entropy of content and subject: The entropy of the email content and subject are 5.25 and 3.88 respectively, indicating that the information is complex and messy, and may attempt to confuse the recipient and make it difficult for them to judge the authenticity of the email. \n Inconsistent information: The email's title and content length and content complexity are inconsistent with legitimate CNN emails. This inconsistency is also one of the common characteristics of phishing emails. \n Malicious Domain Generation Algorithm (DGA): The email was identified as using a malicious domain generation algorithm (DGA), which is a common phishing and malware strategy used to generate a large number of random domain names to avoid security detection. \n This is the first template."},
        {"role": "assistant", "content": "I have already known the first template, please provide me with the second template."},
        {"role": "user", "content": "The second template is in the following. \n Inputs: \n Email Content: \n Ralf Hildebrandt:\n> In the docs I read:\n> \n> Postfix 2.3.x supports Milter Protocol 8 3 and 4\n> Postfix 2.5.x supports Milter Protocol 8 up to 6\n\nMILTER_README says:\n\n    The default version is 2. Other protocol versions are 3 and 4\n    (Postfix 2.3 and later), and 6 (Postfix 2.5 an later).\n\nI report only when features are added or removed; I do not report\nthe gazillion features that releases have in common.\n\n> * Where's the difference anyway?\n\nEach protocol version (the number that is sent in the initial\nPostfix-to-Milter handshake) introduces some new features. The\nMilter API documentation sits in the Sendmail source code tree.\nIt is not reproduced with Postfix.\n\nTo avoid inter-operability problems, Postfix will only announce\nsupport for the features that exist in the configured protocol\nversion.\n\n\tWietse \n Email Subject: Re: Milter Version clarification \n Subject Length: 32 \n Content Length: 821 \n Special Symbol Number: 16 \n Average Word Length per Sentence: 4.99 \n URL Average Length: 0.0 \n URL Belong to Top Website: True \n IS DGA: False \n Sentiment Score: Positive \n Number scarcity: 0 \n Number authority: 3 \n Number urgency: 0 \n Number curiosity: 1 \n Number familiarity: 0 \n Number emotional: 0 \n Number sensitive: 0 \n Number social proof: 0 \n Subject Entropy: 3.82 \n Content Entropy: 4.67 \n\n Outputs: \n Email Type: Benign \n Brand Name: Postfix (associated with Milter Protocol) \n Brand Category: Email Systems/Technology \n Explanation: \n Average URL length: There are no URLs present in the email, which reduces the risk of phishing links. The absence of URLs is consistent with the nature of technical discussions, often focused on configuration or protocol details. \n URLs belong to top-level websites: Not applicable as there are no URLs. \n Abnormal URL format: Not applicable in this case due to the absence of URLs. \n Positive sentiment score: The email has a positive sentiment score, reflecting a neutral or informative tone common in technical discussions. \n No special symbols: The email contains a few special symbols (16), which appear to be standard for technical communication, such as indicating version numbers or protocols. \n Entropy of content and subject: The entropy values for both the content (4.67) and subject (3.82) suggest a moderate level of complexity. This is expected in technical communications discussing version differences and compatibility issues. \n Inconsistent information: The information presented in the email is consistent with a technical discussion regarding different versions of Milter protocols supported by Postfix. It includes factual details and clarifications, which are typical in such exchanges. \n Contextual relevance: The email is contextually relevant to those involved in email systems and protocol management. It provides clarification on the Milter version supported by different versions of Postfix, which is useful information for system administrators and developers. \n This is the second template."},
        {"role": "assistant", "content": "Based on the two templates you provided, I will analyze a new email and determine whether it is a phishing email or not."},
        {"role": "user", "content": "Then I will provide you with new inputs. Please strictly follow the two templates that I give you to generate the new outputs. The outputs should contain the following four elements: \'Email Type:\', \'Brand Name:\', \'Brand Category:\', and \'Explanation:\'. Ensure that all four elements are included in the output, separated by different lines, and do not change their names. The most important thing is that for the first element \'Email Type:\', the output should be only one word from Benign or Phishing or Unknown, and all the outputs should be behind the colons."},
        {"role": "assistant", "content": "Sure, I will follow the two templates you provided strictly and generate the outputs for the new inputs. Please give me the new inputs."},
        {"role": "user", "content": new_input_3 + "Please provide me with the new outputs with four elements!"},
    ]

    model_inputs = tokenizer_1.apply_chat_template(messages, return_tensors="pt").to("cuda")
    generated_ids = model_1.generate(model_inputs, max_new_tokens=200, do_sample=True)

    generated_text_1 = tokenizer_1.batch_decode(generated_ids)
    real_text_1 = generated_text_1[0]

    start_index = real_text_1.rfind("Please provide me with the new outputs with four elements!") + len("Please provide me with the new outputs with four elements! [/INST] ")
    final_content = real_text_1[start_index:]

    output_string.append(real_text_1)
    index_list.append(start_index)

    print(acc)

    check_value = helper_function_01(final_content, new_dataset_dictionary, acc, log_dictionary, acc_2)

    if check_value == True:
        acc+=1
        acc_2 = 0

    if check_2 == True:
        print(final_content)
        check_2 = False

    if check_value == False:
        print(final_content)
        check_2 = True
        acc_2 += 1



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

296


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


297


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


298
299


In [ ]:
print(len(new_dataset_dictionary["Email Type"]))
print(len(new_dataset_dictionary["Brand Name"]))
print(len(new_dataset_dictionary["Brand Category"]))
print(len(new_dataset_dictionary["Explanation"]))

300
300
300
300


In [ ]:
print(new_dataset_dictionary['Email Type'])
print(new_dataset_dictionary['Brand Name'])
print(new_dataset_dictionary['Brand Category'])
print(new_dataset_dictionary['Explanation'])

['Benign', 'Phishing', 'Benign', 'Unknown', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Unknown', 'Phishing', 'Phishing', 'Benign', 'Phishing', 'Unknown', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Unknown', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Phishing', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign',

In [ ]:
log_dataset = pd.DataFrame(log_dictionary)
log_dataset

,Number,First Element,Second Element,Third Element,Fourth Element
0,0,Successful,Successful,Successful,Successful
1,1,Successful,Successful,Successful,Successful
2,2,Successful,Successful,Successful,Successful
3,3,Successful,Successful,Successful,Successful
4,4,Successful,Successful,Successful,Successful
...,...,...,...,...,...
295,295,Successful,Successful,Successful,Successful
296,296,Successful,Successful,Successful,Successful
297,297,Successful,Successful,Successful,Successful
298,298,Successful,Successful,Successful,Successful


In [ ]:
log_dataset_name = 'log' + str(begin_number) + '.csv'

log_dataset = log_dataset.iloc[:, :]
log_dataset.to_csv(log_dataset_name, index=False)

In [ ]:
df = pd.DataFrame(new_dataset_dictionary)
df = df.iloc[:, :]
df

,Email Type,Brand Name,Brand Category,Explanation
0,Benign,Postfix,Email Systems/Technology,The email is a technical discussion concerning...
1,Phishing,flapprice.com,Unknown,The email contains a phishing link to the site...
2,Benign,LREC Workshop,Academic Conference,"Benign email discussing the ""LREC Workshop: Di..."
3,Unknown,N/A,N/A,The email type could not be determined with th...
4,Benign,Equistar,Energy,"The email contains no URLs, thus reducing the ..."
...,...,...,...,...
295,Phishing,National Lottery,Lottery,"* The email contains a large sum of money won,..."
296,Benign,OpenSUSE,Linux Distribution,The email does not contain any suspicious link...
297,Benign,VIA EPIA,Hardware,* Email Type: The email content does not conta...
298,Benign,"Postfix, dkim-milter","Email Systems, Technology","Email type is determined as Benign, based on t..."


In [ ]:
output_dataset = df

In [ ]:
output_dataset_name = str(begin_number) + '.csv'
output_dataset.to_csv(output_dataset_name, index=False)

In [ ]:
output_dataset

,Email Type,Brand Name,Brand Category,Explanation
0,Benign,Postfix,Email Systems/Technology,The email is a technical discussion concerning...
1,Phishing,flapprice.com,Unknown,The email contains a phishing link to the site...
2,Benign,LREC Workshop,Academic Conference,"Benign email discussing the ""LREC Workshop: Di..."
3,Unknown,N/A,N/A,The email type could not be determined with th...
4,Benign,Equistar,Energy,"The email contains no URLs, thus reducing the ..."
...,...,...,...,...
295,Phishing,National Lottery,Lottery,"* The email contains a large sum of money won,..."
296,Benign,OpenSUSE,Linux Distribution,The email does not contain any suspicious link...
297,Benign,VIA EPIA,Hardware,* Email Type: The email content does not conta...
298,Benign,"Postfix, dkim-milter","Email Systems, Technology","Email type is determined as Benign, based on t..."


In [ ]:
print(error_number_list)

[]


In [ ]:
print(len(output_string))

300


In [ ]:
print(output_string[-1][11307:])

The new inputs is coming: 
 Email Content: -----BEGIN PGP SIGNED MESSAGE-----
Hash: SHA1



The Tuesday 2008-02-19 at 09:29 +0100, Per Jessen wrote:

> Aaron Kulkis wrote:
>
>>> But it forces to repartition the entire drive, and I don't feel LVM
>>> is as safe.
>>
>> It's not, which is why I don't use it.
>>
>> All LVM partitions rely on the filesystem holding the LVM table
>> to remain uncorrupted.  Corruption on *THAT* filesystem can
>> very well destroy your LVM partitioning. :-/
>
> Any other suggestions for something that will let me expand my
> filesystem on my systems that must be kept running 365x24?
>
> LVM and EVMS are both actually used quite a lot because they solve a
> real problem. Similar systems have been in use in e.g. AIX and HPUX for
> over 10 years.

That's true, but it is not a solution for those of us having twenty 
physical partitions already in a drive.

Every engineering solution has pros and cons. LVM is good for something, 
physical partitions is good for ano

In [ ]:
print(output_string[182][index_list[182]:])

Email Type: Benign
Brand Name: CNN
Brand Category: News

Explanation:
The email is from CNN and contains the daily top 10 videos and stories. The URLs in the email lead to legitimate CNN webpages. The content is of informational nature and is not trying to solicit personal information or convince the reader to take any unfamiliar actions. There are no special symbols or emotional language used to intimidate or manipulate the reader. The email is not inconsistent with legitimate CNN communication. Therefore, the email is considered to be benign.

Additionally, the email has an average word length per sentence of 11.08, an average URL length of 79.20833333333333 and an IS DGA score of True, but these factors were not used in the determination of the email type due to their lack of relevance based on the information provided


## This is the end of formal code. The following is just for test purpose.

In [ ]:
# Keep this. Do not delete them.

messages = [
        {"role": "user", "content": "I want to use a Large Language Model (LLM) to detect phishing emails. In the next some prompts, I will provide four template examples that includes both input and output. Using these four templates, I will then provide new inputs for which you need to generate the corresponding outputs, identifying whether this input is a phishing email or not based on the template provided."},

        {"role": "assistant", "content": "Sure, I'll do my best to assist you in detecting phishing emails using the Large Language Model (LLM). Please provide these four template examples, including both input and output, and I will generate the corresponding outputs for new inputs based on these templates. \n For the input, please provide a sample email that you would like me to analyze, and for the output, please indicate whether the email is a phishing attempt or not, and explain the reasons for my classification. \n Once you provide the four templates, I will follow your format and generate the corresponding outputs for new inputs that you provide."},

        {"role": "user", "content": "The first template is in the following. \n Inputs: \n Email Content: >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+= >THE DAILY TOP 10 >from CNN.com >Top videos and stories as of: Aug 1, 2008 3:58 PM EDT >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+= TOP 10 VIDEOS 1. MONTAUK 'MONSTER' http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/07/31/moos.montauk.monster.cnn Is it a devil dog? Is it a turtle? Is it the Montauk Monster? CNN's Jeanne Moos asks, \"what is this thing?\" 2. RACY PHOTOS OF TODDLER'S MOM http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/07/31/ng.racy.photos.cnn 3. NEWS OF THE ABSURD EPISODE 54 http://www.cnn.com/video/partners/email/index.html?url=/video/podcasts/absurd/site/2008/08/01/nota.episode.54.cnn 4. POLICE BEATING DISPUTE http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/07/31/levs.police.video.cnn 5. MOM PLEADS FOR GIRL'S RETURN http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/08/01/hill.boss.reigh.plea.cnn 6. DEFENDANT FAKES HEART ATTACK http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/08/01/dnt.fake.heart.attack.mxf.whio 7. KILLER CARRIED VICTIM'S HEAD http://www.cnn.com/video/partners/email/index.html?url=/video/world/2008/07/31/natpkg.can.bus.decapitation.ctv 8. MURDER CONFESSION RECANTED http://www.cnn.com/video/partners/email/index.html?url=/video/world/2008/08/01/blake.brazil.teen.murder.itn 9. ANTHRAX SUSPECT'S HOME http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/08/01/von.dr.bruce.ivins.home.cnn 10. HECKLERS INTERRUPT OBAMA TALK http://www.cnn.com/video/partners/email/index.html?url=/video/politics/2008/08/01/sot.fl.obama.protesters.baynews9 TOP 10 STORIES 1. SUSPECT IN BEHEADING IDENTIFIED http://www.cnn.com/2008/WORLD/americas/08/01/canada.beheading/index.html Canadian police say Vince Weiguang Li, 40, of Edmonton is charged with second-degree murder in the beheading of a man on a bus. 2. JUDGE TAKEN OFF LAST JENA 6 CASES http://www.cnn.com/2008/CRIME/08/01/jena6.appeal/index.html 3. PEOPLE MAG GETS PITT-JOLIE PIX http://www.cnn.com/2008/SHOWBIZ/Movies/08/01/brangelina.photos.ap/index.html 4. ATTACK IN TORONTO CALLED RACIAL http://www.cnn.com/2008/WORLD/americas/08/01/canada.attack.ap/index.html 5. IREPORTERS' UNUSUAL NAMES http://www.cnn.com/2008/LIVING/08/01/unusual.names.irpt/index.html 6. MOTHER PLEADS FOR CHILD'S RETURN http://www.cnn.com/2008/CRIME/08/01/rockefeller.kidnapping/index.html 7. KARADZIC: I MADE DEAL WITH U.S. http://www.cnn.com/2008/WORLD/europe/08/01/karadzic.trial/index.html 8. SUSPECT ARRESTED IN SWIM KILLINGS http://www.cnn.com/2008/CRIME/08/01/wisconsin.shooting.ap/index.html 9. ANTHRAX SUSPECT APPARENT SUICIDE http://www.cnn.com/2008/CRIME/08/01/anthrax.death/index.html 10. MCCAIN: OBAMA CRITICISM 'FAIR' http://www.cnn.com/2008/POLITICS/08/01/campaign.wrap/index.html CNN, The Most Trusted Name in News > Cable News Network LP, LLLP. < > One CNN Center, Atlanta, Georgia 30303 < > 2008 Cable News Network LP, LLLP. < > A Time Warner Company. < > All Rights Reserved. < ========================================================= = Please send comments or suggestions by going to = = http://www.cnn.com/feedback/ = = = = Read our privacy guidelines by going to = = http://www.cnn.com/privacy.html = ========================================================= You have agreed to receive this email from CNN.com as a result of your CNN.com preference settings. To manage your settings, go to: http://www.cnn.com/linkto/bn.manage.html To unsubscribe from the Daily Top 10, go to http://cgi.cnn.com/m/clik?e=email118@gvc.ceas-challenge.cc&l=cnn-dailytop10 \n Email Subject: CNN.com Daily Top 10 \n Subject Length: 20 \n Content Length: 3894 \n Special Symbol Number: 292 \n Average Word Length per Sentence: 11.08 \n URL Average Length: 79.17 \n URL Belong to Top Website: False \n IS DGA: True \n Sentiment Score: Negative \n Number scarcity: 0 \n Number authority: 11 \n Number urgency: 0 \n Number curiosity: 5 \n Number familiarity: 0 \n Number emotional: 1 \n Number sensitive: 0 \n Number social proof: 0 \n Subject Entropy: 3.88 \n Content Entropy:5.25 \n Outputs: \n Email Type (Phishing, Benign, Unknown): Phishing Email \n Brand Name (Unknown of specific name): CNN \n Brand Category: News \n Explanation: \n Average URL length: The average length of URLs included in the emails was 79.17, which is unusually long. Generally speaking, legitimate URLs are usually shorter, especially those from well-known websites. \n URLs do not belong to top-level websites: Although the email looks like it comes from CNN, the URLs in it do not belong to CNN's top-level website domain. This is a major red flag and may indicate that these links are fake and designed to trick users into clicking and leaking sensitive information. \n Abnormal URL format: The URLs in the email contain many complex paths and parameters, which is not common in legitimate emails. Such complex URLs are often used to hide malicious code or redirect to phishing websites. \n Negative sentiment score: The email has a negative sentiment score, indicating that its content may induce negative emotions, such as fear or urgency. Phishing emails often use this strategy to lure recipients into clicking malicious links. \n Unusual number of symbols: The email contains 292 special symbols, which is much more than normal emails. This may be to circumvent spam filters or make the email look more formal and complex, thereby increasing credibility. \n Entropy of content and subject: The entropy of the email content and subject are 5.25 and 3.88 respectively, indicating that the information is complex and messy, and may attempt to confuse the recipient and make it difficult for them to judge the authenticity of the email. \n Inconsistent information: The email's title and content length and content complexity are inconsistent with legitimate CNN emails. This inconsistency is also one of the common characteristics of phishing emails. \n Malicious Domain Generation Algorithm (DGA): The email was identified as using a malicious domain generation algorithm (DGA), which is a common phishing and malware strategy used to generate a large number of random domain names to avoid security detection. \n This is the first template."},

        {"role": "assistant", "content": "I have already known the first template, please provide me with the second template."},

        {"role": "user", "content": "The second template is in the following. \n Inputs: \n Email Content: \n Ralf Hildebrandt:\n> In the docs I read:\n> \n> Postfix 2.3.x supports Milter Protocol 8 3 and 4\n> Postfix 2.5.x supports Milter Protocol 8 up to 6\n\nMILTER_README says:\n\n    The default version is 2. Other protocol versions are 3 and 4\n    (Postfix 2.3 and later), and 6 (Postfix 2.5 an later).\n\nI report only when features are added or removed; I do not report\nthe gazillion features that releases have in common.\n\n> * Where's the difference anyway?\n\nEach protocol version (the number that is sent in the initial\nPostfix-to-Milter handshake) introduces some new features. The\nMilter API documentation sits in the Sendmail source code tree.\nIt is not reproduced with Postfix.\n\nTo avoid inter-operability problems, Postfix will only announce\nsupport for the features that exist in the configured protocol\nversion.\n\n\tWietse \n Email Subject: Re: Milter Version clarification \n Subject Length: 32 \n Content Length: 821 \n Special Symbol Number: 16 \n Average Word Length per Sentence: 4.99 \n URL Average Length: 0.0 \n URL Belong to Top Website: True \n IS DGA: False \n Sentiment Score: Positive \n Number scarcity: 0 \n Number authority: 3 \n Number urgency: 0 \n Number curiosity: 1 \n Number familiarity: 0 \n Number emotional: 0 \n Number sensitive: 0 \n Number social proof: 0 \n Subject Entropy: 3.82 \n Content Entropy: 4.67 \n Outputs: \n Email Type (Phishing, Benign, Unknown): Benign \n Brand Name (Unknown of specific name): Postfix (associated with Milter Protocol) \n Brand Category: Email Systems/Technology \n Explanation: \n Average URL length: There are no URLs present in the email, which reduces the risk of phishing links. The absence of URLs is consistent with the nature of technical discussions, often focused on configuration or protocol details. \n URLs belong to top-level websites: Not applicable as there are no URLs. \n Abnormal URL format: Not applicable in this case due to the absence of URLs. \n Positive sentiment score: The email has a positive sentiment score, reflecting a neutral or informative tone common in technical discussions. \n No special symbols: The email contains a few special symbols (16), which appear to be standard for technical communication, such as indicating version numbers or protocols. \n Entropy of content and subject: The entropy values for both the content (4.67) and subject (3.82) suggest a moderate level of complexity. This is expected in technical communications discussing version differences and compatibility issues. \n Inconsistent information: The information presented in the email is consistent with a technical discussion regarding different versions of Milter protocols supported by Postfix. It includes factual details and clarifications, which are typical in such exchanges. \n Contextual relevance: The email is contextually relevant to those involved in email systems and protocol management. It provides clarification on the Milter version supported by different versions of Postfix, which is useful information for system administrators and developers. \n This is the second template."},

        {"role": "assistant", "content": "I have already known the second template, please provide me with the third template."},

        {"role": "user", "content": "The third template is in the following. \n Inputs: \n Email Content: \n URL: http://boingboing.net/#85531557\nDate: Not supplied\n\nDisney\'s themepark business is in deep trouble in the post-9/11 world. A war \nwith Iraq could really kill \'em: \n\n    While aggressively adding attractions, Disney boosted its profit by \n    steadily raising admission prices. The strategy worked, helping deliver \n    record profit for Disney year after year. \n\n  \"The strategy was build, build, build. Every year there was something new,\" \n    said David Koenig, a Disney historian and author. \"It was an astounding \n    growth period.... Now they\'re overexposed.\"... \n\n    Theme park operating income for Disney this year is expected to fall 27% to \n    $1.16 billion, said Prudential Securities analyst Katherine Styponias. By \n    2003, she said, the business could climb to $1.49 billion, depending on \n    whether the U.S. goes to war with Iraq.  \n\nLink[1] Discuss[2]\n\n[1] http://www.latimes.com/business/la-fi-disney6oct06,0,1287712.story?coll=la%2Dheadlines%2Dbusiness%2Dmanual\n[2] http://www.quicktopic.com/boing/H/iewxPJy39cF7 \n Email Subject: 9/11, war in Iraq threaten Disney parks \n Subject Length: 39 \n Content Length: 1044 \n Special Symbol Number: 12 \n Average Word Length per Sentence: 6.76 \n URL Average Length: 58.0 \n URL Belong to Top Website: False \n IS DGA: False \n Sentiment Score: Positive \n Number scarcity: 0 \n Number authority: 0 \n Number urgency: 0 \n Number curiosity: 2 \n Number familiarity: 1 \n Number emotional: 1 \n Number sensitive: 0 \n Number social proof: 0 \n Subject Entropy: 4.03 \n Content Entropy:4.89 \n Outputs: \n Email Type (Phishing, Benign, Unknown): Benign \n Brand Name (Unknown of specific name): Disney \n Brand Category: Entertainment/Theme Parks \n Explanation: \n Average URL length: The average URL length in the email is 58.0, which is within a typical range and does not raise immediate concerns about abnormal URL patterns. \n URLs do not belong to top-level websites: Although the URLs do not belong to a top-level website, they are associated with reputable sources, such as the Los Angeles Times. This indicates that the content may be sourced from legitimate news outlets. \n Abnormal URL format: The URLs do not exhibit any abnormal or suspicious characteristics. They appear to be standard links to articles and discussions. \n Positive sentiment score: The sentiment score is positive, which aligns with the content’s informative nature, discussing financial impacts on Disney’s business. \n No special symbols: The email contains a few special symbols (12), but they do not significantly contribute to the perception of the email being suspicious. \n Entropy of content and subject: The entropy values for both the content (4.89) and subject (4.03) indicate a moderate level of complexity. This is expected in emails discussing financial data and market analyses. \n Inconsistent information: The information presented in the email appears consistent with news reporting and analysis, which is typical for a benign email. \n Contextual relevance: The email discusses the impact of geopolitical events on a major company’s business, which is a relevant topic that does not inherently suggest phishing. \n This is the third template."},

        {"role": "assistant", "content": "I have already known the third template, please provide me with the last template."},

        {"role": "user", "content": "The fourth template is in the following. \n Inputs: \n Email Content: \n Qualitative watches at Replica Classics \n\nIf you want a decent watch, get a replica approach. ...   \n\nBest Frank Muller replica watches at Replica Classics \n\nhttp://dryadsnedsjaw.com/ \n Email Subject: If you want a decent watch, get a replica approach. ...  \n Subject Length: 58 \n Content Length: 187 \n Special Symbol Number: 0 \n Average Word Length per Sentence: 6.04 \n URL Average Length: 25.0 \n URL Belong to Top Website: False \n IS DGA: False \n Sentiment Score: Positive \n Number scarcity: 0 \n Number authority: 0 \n Number urgency: 0 \n Number curiosity: 0 \n Number familiarity: 0 \n Number emotional: 0 \n Number sensitive: 0 \n Number social proof: 0 \n Subject Entropy: 3.88 \n Content Entropy:4.43 \n Outputs: \n Email Type (Phishing, Benign, Unknown): Phishing Email \n Brand Name (Unknown of specific name): Replica Classics \n Brand Category: Watches \n Explanation: \n Average URL length: The average length of the URL included in the email is 25.0, which is relatively short and unusual for legitimate URLs, especially from reputable websites. \n URLs do not belong to top-level websites: The URL in the email does not belong to a well-known or trusted website. This is a significant indicator of a potential phishing attempt, as phishers often use obscure or unfamiliar domains to deceive recipients. \n Abnormal URL format: The URL provided is not from a recognizable or legitimate source, which suggests it may be used to mislead recipients into visiting potentially malicious sites. \n Positive sentiment score: Despite the positive sentiment score, which might be used to lure recipients into feeling comfortable or intrigued, the nature of the email promoting replica watches raises red flags about its authenticity and legitimacy. \n No special symbols: The absence of special symbols is not common in phishing emails, but it does not negate the other indicators of a phishing attempt present in this email. \n Entropy of content and subject: The entropy values for both content (4.43) and subject (3.88) indicate a moderate level of complexity and randomness, which might be used to create a sense of legitimacy or intrigue. \n Inconsistent information: The promotion of replica watches, a common subject for fraudulent schemes, along with the other indicators, supports the assessment of this email as a phishing attempt. \n This is the fourth template."},

        {"role": "assistant", "content": "Based on the four templates you provided, I will analyze a new email and determine whether it is a phishing email or not."},

        {"role": "user", "content": "Then I will provide you with new inputs. Please strictly follow the four templates that I give you to generate the new outputs. The outputs should contain the following four elements: Email Type (Phishing, Benign, Unknown), Brand Name (Unknown or specific name), Brand Category, and Explanation. Ensure that all four elements are included in the output, and do not change their names."},

        {"role": "assistant", "content": "Sure, I will follow the four templates you provided strictly and generate the outputs for the new inputs. Please give me the new inputs."},

        {"role": "user", "content": new_input_3 + "Please provide me with the new outputs with four elements!"},
    ]



In [ ]:
dataset["sentiment_score"] = dataset["sentiment_score"].apply(lambda x: 'Positive' if x > 0 else 'Negative')
dataset["average_word_length"] = dataset["average_word_length"].round(2)
dataset["subject_entropy"] = dataset["subject_entropy"].round(2)
dataset["body_entropy"] = dataset["body_entropy"].round(2)

dataset

TypeError: '>' not supported between instances of 'str' and 'int'